In [1]:
import pandas as pd
import numpy as np

# combined data - 2 tables

In [2]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [3]:
filename_leadercareerlink = "leadercareerlink.xlsx"

In [4]:
filename_leaderjoblink = "leaderjoblink.xlsx"

In [5]:
filename_joborglink = "joborglink.xlsx"

# combined data - 1 cleaning/cleaning step 2 - career/

In [6]:
path_cleaning = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/"

In [87]:
subpath_2_1 = "2.1 career_undivided_unparsed_uncoded/"
subpath_2_2 = "2.2 career_divided_unparsed_uncoded/"
subpath_2_3 = "2.3 joborglink/"
subpath_2_6 = "2.6 career_reassembled/"

In [8]:
filein_careerlink = "career_link_undivided_unparsed_uncoded.xlsx"
careerlink = pd.read_excel(path_cleaning + subpath_2_1 + filein_careerlink)
careerlink.shape

(12985, 4)

In [9]:
filein_careerjobs = "career_jobs_unparsed_uncoded.xlsx"
careerjobs = pd.read_excel(path_cleaning + subpath_2_2 + filein_careerjobs)
careerjobs.shape

(7590, 3)

In [10]:
filein_singlesub = "joborglink_2_substr_single_1tocode_4Mail_Lee.xlsx"
singlesub = pd.read_excel(path_cleaning + subpath_2_3 + filein_singlesub)
singlesub.shape

(5896, 9)

In [11]:
# hand coded MultipleSubstrings, NotJob
# update these variables in careers
# but [CareerString,CareerDateString_2022] are mismatched
# ignore CareerDateString_2022. only match on CareerString

filein_multiple = "joborglink_1_mergeold_mail.xlsx"
jol1 = pd.read_excel(path_cleaning + subpath_2_3 + filein_multiple)
jol1.shape

(7592, 14)

In [12]:
# multisub should have 1695 unique CareerString. 1756 are the expanded CareerSubstrings
filein_multisub = "joborglink_2_substr_multi2_1tocode_Mail_Lee.xlsx"
multisub = pd.read_excel(path_cleaning + subpath_2_3 + filein_multisub)
multisub.shape

(1756, 11)

In [13]:
filein_jol_7 = "joborglink_7_2.xlsx"
jol7 = pd.read_excel(path_cleaning + subpath_2_3 + filein_jol_7)
jol7.shape

(1425, 4)

# 1. leadercareerlink 

In [14]:
# from career_link_undivided_unparsed_uncoded
careerlink.head(3)

,LeaderID,CareerStringID_2015,CareerString,CareerDateString_2022
0,리선권,NaN,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,NaN,故 김정일 국가장의위원회 위원,2011.12
2,리영수,NaN,근로단체부장 해임 * 후임: 리일환,2014


In [15]:
select_columns = ["LeaderID","CareerString","CareerDateString_2022"]
careerlink = careerlink[select_columns]

In [16]:
careerlink[careerlink.duplicated(keep=False)]

,LeaderID,CareerString,CareerDateString_2022
241,김일본,1950.12 민족보위성 부상 겸 문화부 부장,NaN
242,김일본,1950.12 민족보위성 부상 겸 문화부 부장,NaN
247,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
248,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
249,박송봉,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",NaN
...,...,...,...
5468,현영철,"2012. 7 인민군 총참모장, 인민군 차수, 당 중앙군사위원회 부위원장",NaN
5488,리철만,2013. 4 내각 부총리 겸 농업상,NaN
5489,리철만,2013. 4 내각 부총리 겸 농업상,NaN
5490,리무영,2013. 4 내각 부총리 겸 화학공업상,NaN


In [17]:
leadercareerlink = careerlink.drop_duplicates(keep="first")
leadercareerlink.shape

(12617, 3)

In [18]:
leadercareerlink.head(5)

,LeaderID,CareerString,CareerDateString_2022
0,리선권,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,故 김정일 국가장의위원회 위원,2011.12
2,리영수,근로단체부장 해임 * 후임: 리일환,2014
3,리설주,금수산궁전 개관식 참석시 동행,2013.08
4,리원일,노동성 상(유임),1999.02


In [19]:
# export leadercareerlink to tables
# leadercareerlink.to_excel(path_tables+filename_leadercareerlink,index=False)

# 2. joborglink

### 2.1 - careers - isolate careers from leadercareerlink

In [20]:
select_columns = ["CareerString","CareerDateString_2022"]
careers = leadercareerlink[select_columns]
careers.shape

(12617, 2)

In [21]:
careers[careers.duplicated(keep=False)].shape

(5125, 2)

In [22]:
careers = careers.drop_duplicates(keep="first")
careers.shape

(8045, 2)

In [23]:
careers[careers.duplicated(["CareerString","CareerDateString_2022"],keep=False)]

,CareerString,CareerDateString_2022


### 2.2 - careers0 - add IsJob
- compare careers with career_jobs_unparsed_uncoded
- add IsJob=False for all rows
- recode IsJob=True for all CareerString contained in career_jobs_unparsed_uncoded
- don't code CareerStringType - because we still have MultipleSubstrings
- just note that all career_jobs_unparsed_uncoded are contained in careers
- later, use the files in subpath_2_2 = "2.2 career_divided_unparsed_uncoded/" to help code CareerSubstringType

In [24]:
# compare careers with careerjobs
careerjobs.head(3)

,CareerString,CareerDateString_2022,CareerStringID
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,1
1,근로단체부장 해임 * 후임: 리일환,2014,2
2,노동성 상(유임),1999.02,4


In [25]:
select_columns = ["CareerString","CareerDateString_2022"]
careerjobs = careerjobs[select_columns]
careerjobs.shape

(7590, 2)

In [26]:
careerjobs[careerjobs.duplicated(keep=False)]

,CareerString,CareerDateString_2022


In [27]:
careers.head(3)

,CareerString,CareerDateString_2022
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,故 김정일 국가장의위원회 위원,2011.12
2,근로단체부장 해임 * 후임: 리일환,2014


In [28]:
# merge and compare careers & careerjobs
select_columns = ["CareerString","CareerDateString_2022"]
careers0 = careers.merge(careerjobs,on=select_columns,how="outer",indicator=True)

In [29]:
careers0.shape

(8045, 3)

In [30]:
careers0.head(3)

,CareerString,CareerDateString_2022,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,both
1,故 김정일 국가장의위원회 위원,2011.12,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,both


In [31]:
careers0[careers0["_merge"]=="left_only"].shape

(455, 3)

In [32]:
careers0[careers0["_merge"]=="both"].shape

(7590, 3)

In [33]:
careers0[careers0["_merge"]=="right_only"].shape

(0, 3)

In [34]:
careers0[careers0["_merge"]=="right_only"]

,CareerString,CareerDateString_2022,_merge


In [35]:
careers0["IsJob"] = False

In [36]:
careers0.loc[careers0["_merge"]=="both","IsJob"] = True

In [37]:
# 7590 jobs
# 455 not jobs
careers0.groupby("IsJob").count()

,CareerString,CareerDateString_2022,_merge
IsJob,,,
False,455,259,455
True,7590,4209,7590


In [38]:
select_columns = ["CareerString","CareerDateString_2022","IsJob"]
careers0 = careers0[select_columns]

In [39]:
careers0.head(3)

,CareerString,CareerDateString_2022,IsJob
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True
1,故 김정일 국가장의위원회 위원,2011.12,False
2,근로단체부장 해임 * 후임: 리일환,2014,True


### 2.2.5 - update CareerStringType for NotJob, using 2.2 career_divided_unparsed_uncoded

In [40]:
# not urgent. do this later. or do it again by hand

### 2.3 - careers1 - add MultipleString
- compare careers0 with jol1 (joborglink_1_mergeold_mail)
 - ignore CareerDateString_2022 and only merge on CareerString because CareerDateString_2022 is incorrect
- update IsJob with jol1["NotJob"]
- add MultipleSubstrings
- code MultipleSubstrings

In [41]:
select_columns = ["CareerString","NotJob","MultipleSubstrings"]
jol1 = jol1[select_columns]
jol1.head(3)

,CareerString,NotJob,MultipleSubstrings
0,"1992.12 정무원 총리 해임, 당 정치국 후보위원으로 강등, 자강도 당 책임비서...",NaN,2.0
1,2003. 3 조선일본군위안부 및 강제연행피해자보상대책위원장,NaN,NaN
2,"당 정치국 위원, 당 중앙위원회 부위원장, 당 국제부장 등 해임(추정)",0.0,NaN


In [42]:
jol1.shape

(7592, 3)

In [43]:
jol1 = jol1.drop_duplicates(keep="first")
jol1.shape

(6659, 3)

In [44]:
# some duplicate CareerStrings with different dates; some are legit, others not
# I removed CareerDateString_2022 from jol1 as it's not critical to updating IsJob & MultipleSubstrings
jol1[jol1.duplicated(["CareerString"],keep=False)].head(10)

,CareerString,NotJob,MultipleSubstrings


In [45]:
# no duplicate keys in jol1
select_key = ["CareerString"]
jol1[jol1.duplicated(select_key,keep=False)]

,CareerString,NotJob,MultipleSubstrings


In [46]:
careers0.shape

(8045, 3)

In [47]:
select_key = ["CareerString"]
careers1 = careers0.merge(jol1,on=select_key,how="outer",indicator=True)
careers1.shape

(8045, 6)

In [48]:
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,True,1.0,NaN,both
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,True,1.0,NaN,both


In [49]:
# 7590 IsJob
# 455 NotJob
careers1.groupby(["_merge"],dropna=False).count()

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings
_merge,,,,,
left_only,455,259,455,0,0
right_only,0,0,0,0,0
both,7590,4209,7590,924,740


In [50]:
careers1[careers1["_merge"]=="right_only"]

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge


In [51]:
# NobJob = 0: 후임, etc.
# NotJob = 1: marriage, education, etc.
# NotJob = NaN: IsJob
careers1.groupby(["IsJob","NotJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  _merge
IsJob NotJob                                                                 
False NaN              455                    259                   0     455
True  0.0              421                    238                   0     421
      1.0              503                    203                   0     503
      NaN             6666                   3768                 740    6666

In [52]:
# if NotJob in [0,1], set IsJob = False
careers1.loc[careers1["NotJob"].isin([0,1]),"IsJob"]=False

In [53]:
# verify successful update of IsJob
careers1.groupby(["IsJob","NotJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  _merge
IsJob NotJob                                                                 
False 0.0              421                    238                   0     421
      1.0              503                    203                   0     503
      NaN              455                    259                   0     455
True  NaN             6666                   3768                 740    6666

In [54]:
careers1.groupby(["MultipleSubstrings"],dropna=False).count()

,CareerString,CareerDateString_2022,IsJob,NotJob,_merge
MultipleSubstrings,,,,,
2.0,574,257,574,0,574
3.0,125,71,125,0,125
4.0,27,15,27,0,27
5.0,14,10,14,0,14
NaN,7305,4115,7305,924,7305


In [55]:
careers1.groupby(["IsJob","MultipleSubstrings"],dropna=False).count()

CareerString  CareerDateString_2022  NotJob  _merge
IsJob MultipleSubstrings                                                     
False NaN                         1379                    700     924    1379
True  2.0                          574                    257       0     574
      3.0                          125                     71       0     125
      4.0                           27                     15       0      27
      5.0                           14                     10       0      14
      NaN                         5926                   3415       0    5926

In [56]:
# accept MultipleSubstrings
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,NotJob,MultipleSubstrings,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,1.0,NaN,both
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,False,1.0,NaN,both


In [57]:
# drop NotJob, _merge
careers1.drop(columns=["NotJob","_merge"],inplace=True)
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN


In [58]:
careers1.groupby(["IsJob","MultipleSubstrings"],dropna=False).count()

CareerString  CareerDateString_2022
IsJob MultipleSubstrings                                     
False NaN                         1379                    700
True  2.0                          574                    257
      3.0                          125                     71
      4.0                           27                     15
      5.0                           14                     10
      NaN                         5926                   3415

### 2.4 - careers2 - add CareerSubstrings from singlesub

- compare careers1 with
 - joborglink_2_substr_single_1tocode_4Mail_Lee *
 - joborglink_2_substr_multi2_1tocode_Mail_Lee
- double-check
 - CareerString: matches careers or jol1 (incomplete)?
 - CareerDateString_2022: matches careers, or jol1 (unreliable)?
- add CareerStartYear, CareerStartMonth, CareerSubstrings

In [59]:
select_columns = ["CareerString","CareerDateString_2022","CareerStartYear","CareerStartMonth","OrgString","PositionString"]
singlesub = singlesub[select_columns]
singlesub.shape

(5896, 6)

In [60]:
singlesub.head(3)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,OrgString,PositionString
0,동만지구 공산주의 청년동맹위원회 서기,1929,1929,NaN,동만지구 공산주의 청년동맹위원회,서기
1,동북항일연군교도여단 제1교도영 영장으로 활동,1942.08,1942,8.0,동북항일연군교도여단 제1교도영,영장
2,소련군 장교,1945.08,1945,8.0,소련군,장교


In [61]:
# 
singlesub[singlesub.duplicated(keep=False)].shape

(42, 6)

In [62]:
singlesub = singlesub.drop_duplicates(keep="first")
singlesub.shape

(5872, 6)

In [63]:
singlesub[singlesub.duplicated(["CareerString","CareerDateString_2022"],keep=False)]

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,OrgString,PositionString


In [64]:
careers1.head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN


In [65]:
singlesub.head(3)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,OrgString,PositionString
0,동만지구 공산주의 청년동맹위원회 서기,1929,1929,NaN,동만지구 공산주의 청년동맹위원회,서기
1,동북항일연군교도여단 제1교도영 영장으로 활동,1942.08,1942,8.0,동북항일연군교도여단 제1교도영,영장
2,소련군 장교,1945.08,1945,8.0,소련군,장교


In [66]:
key_columns = ["CareerString","CareerDateString_2022"]
careers2 = careers1.merge(singlesub,on=key_columns,how="outer",indicator=True)
careers2.head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN,2013,9.0,개성공단 남북공동위원회 통행통신통관 분과위원회,NaN,both
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,NaN,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN,2014,NaN,근로단체,부장,both


In [67]:
careers2.shape

(8045, 9)

In [68]:
careers1.shape

(8045, 4)

In [69]:
# right_only = 0
# left_only, IsJob=True = 900: hopefully contained in multisub
careers2.groupby(["_merge","IsJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  \
_merge     IsJob                                                            
left_only  False          1271                    629                   0   
           True            902                    409                 678   
right_only False             0                      0                   0   
           True              0                      0                   0   
both       False           108                     71                   0   
           True           5764                   3359                  62   

                  CareerStartYear  CareerStartMonth  OrgString  PositionString  
_merge     IsJob                                                                
left_only  False                0                 0          0               0  
           True                 0                 0          0               0  
right_only False                0                 0          0               0  
           True                 0                 0          0               0  
both       False               93                86         91              90  
           True              5424              4784       5695            5687

In [70]:
# for (IsJob=True & MultipleSubstrings=NaN & OrgString=NaN) update IsJob, MultipleSubstrings
# it seems like most of these will be IsJob=False
careers2[(careers2["IsJob"]==True) & (careers2["_merge"]=="left_only")]

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,_merge
10,만경대혁명학원,1952.11 ~ 1953,True,NaN,NaN,NaN,NaN,NaN,left_only
55,*프에블로호 美정보함 피납사건 당시 근무,NaN,True,NaN,NaN,NaN,NaN,NaN,left_only
131,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,True,NaN,NaN,NaN,NaN,NaN,left_only
132,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,True,2.0,NaN,NaN,NaN,NaN,left_only
133,△인민군 총참모장 △인민군 대장,2009.02,True,2.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...
7877,하얼빈공대,1940,True,NaN,NaN,NaN,NaN,NaN,left_only
7888,함경남도 당 위원회 책임비서 겸 인민위원장,1996.01,True,NaN,NaN,NaN,NaN,NaN,left_only
7952,해군 사령관 해임 * 후임: 리용주,2015.04,True,NaN,NaN,NaN,NaN,NaN,left_only
7978,호위사령관 해임,2004,True,NaN,NaN,NaN,NaN,NaN,left_only


In [71]:
careers2.drop(columns=["_merge"],inplace=True)

### 2.5 - careers3 - add CareerSubstrings from multisub

- compare careers1 with
 - joborglink_2_substr_single_1tocode_4Mail_Lee
 - joborglink_2_substr_multi2_1tocode_Mail_Lee *
- double-check
 - CareerString: matches careers or jol1 (incomplete)?
 - CareerDateString_2022: matches careers, or jol1 (unreliable)?
- add CareerStartYear, CareerStartMonth, CareerSubstrings

In [72]:
select_columns = ["CareerString","CareerDateString_2022","CareerStartYear","CareerStartMonth","MultipleSubstrings","CareerSubstring","OrgString","PositionString"]
multisub = multisub[select_columns]
multisub.shape

(1756, 8)

In [73]:
multisub.head(3)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstring,OrgString,PositionString
0,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN,NaN,NaN
1,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN,NaN,NaN
2,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,2012,4.0,2,NaN,NaN,NaN


In [74]:
# expanded rows which haven't been coded
multisub[multisub.duplicated(keep=False)].head(5)

,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstring,OrgString,PositionString
0,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN,NaN,NaN
1,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,2009,11.0,2,NaN,NaN,NaN
2,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,2012,4.0,2,NaN,NaN,NaN
3,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,2012,4.0,2,NaN,NaN,NaN
4,△인민군 총참모장 △인민군 대장,2009.02,2009,2.0,2,NaN,NaN,NaN


In [75]:
key_columns = ["CareerString","CareerDateString_2022"]
careers3 = careers2.merge(multisub,on=key_columns,how="outer",suffixes=("","_y"),indicator=True)
careers3.head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,CareerStartYear_y,CareerStartMonth_y,MultipleSubstrings_y,CareerSubstring,OrgString_y,PositionString_y,_merge
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN,2013,9.0,개성공단 남북공동위원회 통행통신통관 분과위원회,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN,2014,NaN,근로단체,부장,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [76]:
careers3.shape

(9026, 15)

In [77]:
careers2.shape

(8045, 8)

In [78]:
careers3.groupby(["_merge"],dropna=False).count()

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,CareerStartYear_y,CareerStartMonth_y,MultipleSubstrings_y,CareerSubstring,OrgString_y,PositionString_y
_merge,,,,,,,,,,,,,,
left_only,7270,4135,7270,74,5517,4870,5786,5777,0,0,0,0,0,0
right_only,0,0,0,0,0,0,0,0,0,0,0,0,0,0
both,1756,788,1756,1537,0,0,0,0,1712,1621,1756,701,694,701


In [79]:
# right_only = 0
careers3.groupby(["_merge","IsJob"],dropna=False).count()

CareerString  CareerDateString_2022  MultipleSubstrings  \
_merge     IsJob                                                            
left_only  False          1370                    696                   0   
           True           5900                   3439                  74   
right_only False             0                      0                   0   
           True              0                      0                   0   
both       False            19                     10                   0   
           True           1737                    778                1537   

                  CareerStartYear  CareerStartMonth  OrgString  \
_merge     IsJob                                                 
left_only  False               93                86         91   
           True              5424              4784       5695   
right_only False                0                 0          0   
           True                 0                 0          0   
both       False                0                 0          0   
           True                 0                 0          0   

                  PositionString  CareerStartYear_y  CareerStartMonth_y  \
_merge     IsJob                                                          
left_only  False              90                  0                   0   
           True             5687                  0                   0   
right_only False               0                  0                   0   
           True                0                  0                   0   
both       False               0                 19                  19   
           True                0               1693                1602   

                  MultipleSubstrings_y  CareerSubstring  OrgString_y  \
_merge     IsJob                                                       
left_only  False                     0                0            0   
           True                      0                0            0   
right_only False                     0                0            0   
           True                      0                0            0   
both       False                    19                0            0   
           True                   1737              701          694   

                  PositionString_y  
_merge     IsJob                    
left_only  False                 0  
           True                  0  
right_only False                 0  
           True                  0  
both       False                 0  
           True                701

In [80]:
# confirmed that IsJob=False multisub are indeed NotJob
careers3[(careers3["IsJob"]==False) & (careers3["_merge"]=="both")]

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,CareerStartYear_y,CareerStartMonth_y,MultipleSubstrings_y,CareerSubstring,OrgString_y,PositionString_y,_merge
2412,2000. 4 인민보안상 2003. 7 인민보안상 해임 *후임 : 최용수,NaN,False,NaN,NaN,NaN,NaN,NaN,2000,4.0,1.0,NaN,NaN,NaN,both
3605,2010. 6 내각 부총리?국가학위학직수여위원회 위원장 해임,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,6.0,2.0,NaN,NaN,NaN,both
3606,2010. 6 내각 부총리?국가학위학직수여위원회 위원장 해임,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,6.0,2.0,NaN,NaN,NaN,both
3615,2010. 6 체육지도위원회 위원장?조선올림픽위원회 위원장 해임 *후임 : 박명철,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,6.0,2.0,NaN,NaN,NaN,both
3616,2010. 6 체육지도위원회 위원장?조선올림픽위원회 위원장 해임 *후임 : 박명철,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,6.0,2.0,NaN,NaN,NaN,both
3675,2010. 9 당 정치국 위원?당 중앙위 위원 해임,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,9.0,2.0,NaN,NaN,NaN,both
3676,2010. 9 당 정치국 위원?당 중앙위 위원 해임,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,9.0,2.0,NaN,NaN,NaN,both
3717,2010. 9 당 중앙위 비서?당 간부부 부장 해임 *후임 : 김평해,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,9.0,2.0,NaN,NaN,NaN,both
3718,2010. 9 당 중앙위 비서?당 간부부 부장 해임 *후임 : 김평해,NaN,False,NaN,NaN,NaN,NaN,NaN,2010,9.0,2.0,NaN,NaN,NaN,both
4206,교육위원회 고등교육상 겸 김일성종합대학 총장 해임 * 후임: 태형철,2013.04,False,NaN,NaN,NaN,NaN,NaN,2013,4.0,2.0,NaN,NaN,NaN,both


In [81]:
# notice that for multisub IsJob=True
# MultipleSubstrings, CareerStartYear, CareerStartMonth, OrgString, PositionString are all NaN
careers3[(careers3["IsJob"]==True) & (careers3["_merge"]=="both")].head(3)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,CareerStartYear_y,CareerStartMonth_y,MultipleSubstrings_y,CareerSubstring,OrgString_y,PositionString_y,_merge
131,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,True,NaN,NaN,NaN,NaN,NaN,2009,11.0,2.0,NaN,NaN,NaN,both
132,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,True,NaN,NaN,NaN,NaN,NaN,2009,11.0,2.0,NaN,NaN,NaN,both
133,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,True,2.0,NaN,NaN,NaN,NaN,2012,4.0,2.0,NaN,NaN,NaN,both


In [82]:
# from multisub, IsJob=True
    # copy over MultipleSubstrings, CareerStartYear, CareerStartMonth, OrgString, PositionString from
careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"MultipleSubstrings"]=careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"MultipleSubstrings_y"]
careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"CareerStartYear"]=careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"CareerStartYear_y"]
careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"CareerStartMonth"]=careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"CareerStartMonth_y"]
careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"OrgString"]=careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"OrgString_y"]
careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"PositionString"]=careers3.loc[(careers3["IsJob"]==True) & (careers3["_merge"]=="both"),"PositionString_y"]

In [83]:
# confirm values have been copied over
careers3[(careers3["IsJob"]==True) & (careers3["_merge"]=="both")].head(4)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,CareerStartYear_y,CareerStartMonth_y,MultipleSubstrings_y,CareerSubstring,OrgString_y,PositionString_y,_merge
131,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,True,2.0,2009,11.0,NaN,NaN,2009,11.0,2.0,NaN,NaN,NaN,both
132,△국토환경보호성 상 △조선자연보호연맹 중앙위원회 위원장 * 전임: 박송남,2009.11,True,2.0,2009,11.0,NaN,NaN,2009,11.0,2.0,NaN,NaN,NaN,both
133,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,True,2.0,2012,4.0,NaN,NaN,2012,4.0,2.0,NaN,NaN,NaN,both
134,△내각 부총리 △수도건설위원회 위원장,2012.04 ~ 2014.04,True,2.0,2012,4.0,NaN,NaN,2012,4.0,2.0,NaN,NaN,NaN,both


In [84]:
# drop columns with _y suffix
careers3.drop(columns=["CareerStartYear_y","CareerStartMonth_y","MultipleSubstrings_y","OrgString_y","PositionString_y","_merge"],inplace=True)

In [85]:
careers3.head(4)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,OrgString,PositionString,CareerSubstring
0,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,False,NaN,2013,9.0,개성공단 남북공동위원회 통행통신통관 분과위원회,NaN,NaN
1,故 김정일 국가장의위원회 위원,2011.12,False,NaN,NaN,NaN,NaN,NaN,NaN
2,근로단체부장 해임 * 후임: 리일환,2014,False,NaN,2014,NaN,근로단체,부장,NaN
3,금수산궁전 개관식 참석시 동행,2013.08,False,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# export in order to verify IsJob, MultipleSubstrings
# careers3.to_excel(path_cleaning + subpath_2_6 + "careers_1_jeongsu.xlsx",index=False)

In [90]:
careers3.groupby(["IsJob","MultipleSubstrings"],dropna=False).count()

CareerString  CareerDateString_2022  \
IsJob MultipleSubstrings                                        
False NaN                         1389                    706   
True  1.0                            3                      3   
      2.0                         1264                    525   
      3.0                          367                    203   
      4.0                          106                     66   
      5.0                           71                     41   
      NaN                         5826                   3379   

                          CareerStartYear  CareerStartMonth  OrgString  \
IsJob MultipleSubstrings                                                 
False NaN                              93                86         91   
True  1.0                               3                 3          0   
      2.0                            1230              1140        622   
      3.0                             349               340        110   
      4.0                             100               100         24   
      5.0                              65                65          0   
      NaN                            5370              4738       5633   

                          PositionString  CareerSubstring  
IsJob MultipleSubstrings                                   
False NaN                             90                0  
True  1.0                              0                0  
      2.0                            626              571  
      3.0                            112              107  
      4.0                             24               23  
      5.0                              0                0  
      NaN                           5626                0

### 2.6 - careers4 - 

